In [ ]:
pip install transformers datasets sacrebleu sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Load OPUS100 English–Punjabi dataset
dataset = load_dataset("opus100", "en-pa")

print(dataset)
print(dataset['train'][21])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

en-pa/test-00000-of-00001.parquet:   0%|          | 0.00/69.3k [00:00<?, ?B/s]

en-pa/train-00000-of-00001.parquet:   0%|          | 0.00/5.18M [00:00<?, ?B/s]

en-pa/validation-00000-of-00001.parquet:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/107296 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 107296
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})
{'translation': {'en': 'Gulbene', 'pa': 'latvia. kgm'}}


In [ ]:
from transformers import M2M100Tokenizer
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer


# Load tokenizer and model
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

# Set source and target language codes
src_lang = "en"
tgt_lang = "pa"
tokenizer.src_lang = src_lang
tokenizer.tgt_lang = tgt_lang

# Preprocessing function (fixed)
def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["pa"] for ex in examples["translation"]]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=64, truncation=True)

    # Tokenize targets with `text_target` (new API, no warnings)
    labels = tokenizer(text_target=targets, max_length=64, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True)
print(tokenized_datasets["train"][0])


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/107296 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': 'Score', 'pa': 'ਅੰਕ'}, 'input_ids': [128022, 7541, 1047, 2], 'attention_mask': [1, 1, 1, 1], 'labels': [128072, 83745, 2411, 2]}


In [ ]:
!pip install -U transformers

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Training arguments with checkpoint saving
training_args = Seq2SeqTrainingArguments(
    output_dir="./checkpoints",          # where checkpoints will be saved
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=200,                   # log every 200 steps
    save_strategy="steps",               # save checkpoints by steps
    save_steps=2000,                     # adjust (every 2000 steps here)
    eval_steps=2000,
    save_total_limit=3,                  # keep last 3 checkpoints only
    num_train_epochs=2,
    fp16=True,
    report_to="none"
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train
trainer.train()

# Save final model
trainer.save_model("my_wikisimplify_model")
tokenizer.save_pretrained("my_wikisimplify_model")


/tmp/ipython-input-2149137932.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
200,2.876500
400,2.355300
600,2.114700
800,1.955800
1000,1.910000
1200,1.831100
1400,1.739800
1600,1.797100
1800,1.664200
2000,1.512200


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4034: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('my_wikisimplify_model/tokenizer_config.json',
 'my_wikisimplify_model/special_tokens_map.json',
 'my_wikisimplify_model/vocab.json',
 'my_wikisimplify_model/sentencepiece.bpe.model',
 'my_wikisimplify_model/added_tokens.json')

In [ ]:
trainer.train(resume_from_checkpoint=True)


In [ ]:
!zip -r my_wikisimplify_model.zip my_wikisimplify_model
from google.colab import files
files.download("my_wikisimplify_model.zip")


  adding: my_wikisimplify_model/ (stored 0%)
  adding: my_wikisimplify_model/model.safetensors (deflated 7%)
  adding: my_wikisimplify_model/added_tokens.json (deflated 76%)
  adding: my_wikisimplify_model/training_args.bin (deflated 54%)
  adding: my_wikisimplify_model/sentencepiece.bpe.model (deflated 50%)
  adding: my_wikisimplify_model/config.json (deflated 57%)
  adding: my_wikisimplify_model/vocab.json (deflated 71%)
  adding: my_wikisimplify_model/tokenizer_config.json (deflated 94%)
  adding: my_wikisimplify_model/generation_config.json (deflated 38%)
  adding: my_wikisimplify_model/special_tokens_map.json (deflated 79%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r my_wikisimplify_model /content/drive/MyDrive/


Mounted at /content/drive
